# Live-stock detection (DeepForest)
## Context
### Purpose
Implement and fine-tune a pre-trained Deep Learning model to detect live-stock in airborne imagery.


### Modelling Approach
The [live-stock detection model](https://deepforest.readthedocs.io/en/latest/prebuilt.html#livestock-detectors) from the latest version (v1.4.0) of the [DeepForest](https://deepforest.readthedocs.io/en/latest/) Deep Learning model is used to predict bounding boxes corresponding to cattle from airborn RGB images.

The prebuilt model was trained on a [limited dataset](https://new.wildlabs.net/discussion/global-model-livestock-detection-airborne-imagery-data-applications-and-needs). According to the package's documentation, "the prebuilt models will always be improved by adding data from the target area". As such, this notebook will explore the improvement in the model's performance in live-stock detection from fine-tuning on local data.

### Description
This notebook will explore the capabilities of the DeepForest package. In particular, it will demonstrate how to:

- Detect live-stock in airborne imagery using the prebuilt live-stock detection model.
- Fine-tune the model using a novel publicly-available dataset.
- Evaluate the the model's performance before and after fine-tuning.

### Highlights
*Provide 3-5 bullet points that convey the use case’s core procedures. Each bullet point must have a maximum of 85 characters, including spaces.*
* Highlight 1
* Highlight 2

### Contributions
#### Notebook
* Cameron Appel (author), Queen Mary University of London, @camappel

#### Modelling codebase
* Ben Weinstein (maintainer & developer), University of Florida, @bw4sz
* Henry Senyondo (support maintainer), University of Florida, @henrykironde
* Ethan White (PI and author), University of Florida, @weecology


## Load libraries
List libraries according to their role e.g. system/files manipulation i.e. os (first), data handling i.e. numpy, xarray (second), visualisation i.e. holoviews (third), etc. The cell below contains two libraries, `os` and `warning` which are common among the Python Jupyter notebooks. Don't remove them.*

In [1]:
!pip -q install torchvision
!pip -q install git+https://github.com/Weecology/DeepForest.git
!pip -q install --upgrade huggingface_hub

In [23]:
import os
import glob
import urllib

import numpy as np
import pandas as pd

import intake
import xmltodict
import cv2
import matplotlib.pyplot as plt

from deepforest import main
from huggingface_hub import hf_hub_download

import torch

from shapely.geometry import box
from skimage.exposure import equalize_hist

import pooch

import warnings
warnings.filterwarnings(action='ignore')

## Set project structure
*The cell below creates a separate folder to save the notebook outputs. This facilitates the reader to inspect inputs/outputs stored within a defined destination folder. Don't remove the lines below.*

In [3]:
notebook_folder = './notebook'
if not os.path.exists(notebook_folder):
    os.makedirs(notebook_folder)

In [4]:
extract_dir = os.path.join(notebook_folder, 'test_data')
if not os.path.exists(extract_dir):
    os.makedirs(extract_dir, exist_ok=True)

# Fetch RGB images from Zenodo

Fetch sample images from Harvard's publicly accessible [ODjAR Dataverse](https://dataverse.harvard.edu/dataverse/ODjAR). 

Specifically, G.J. Franke; Sander Mucher, 2021, "Annotated cows in aerial images for use in deep learning models", which includes "a large dataset containing aerial images from fields in Juchowo, Poland and Wageningen, the Netherlands, with annotated cows present in the images using Pascal VOC XML Annotation Format."

In [13]:
unzipped_files = pooch.retrieve(
    url="doi:10.5281/zenodo.13851270/test_data.zip",
    known_hash="6a0a5b48fc9326e97c3cd8bdcabc2bcd131f3755f6ceabbf6976aefbfc87fb00",
    processor=pooch.Unzip(extract_dir=extract_dir),
    path=f"."
)

In [17]:
# Load the CSV (annotations), assuming it's also part of the unzipped files
test_path = [file for file in unzipped_files if file.endswith('test.csv')][0]
test_df = pd.read_csv(test_path)

# Download baseline model

In [16]:
model = main.deepforest()
model.load_model(model_name="weecology/deepforest-livestock", revision="main")

Reading config file: /Users/cam/miniconda3/envs/deepforest/lib/python3.10/site-packages/deepforest/data/deepforest_config.yml


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Reading config file: /Users/cam/miniconda3/envs/deepforest/lib/python3.10/site-packages/deepforest/data/deepforest_config.yml


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


model.safetensors:   0%|          | 0.00/129M [00:00<?, ?B/s]

deepforest(
  (model): RetinaNet(
    (backbone): BackboneWithFPN(
      (body): IntermediateLayerGetter(
        (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
        (bn1): FrozenBatchNorm2d(64, eps=0.0)
        (relu): ReLU(inplace=True)
        (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
        (layer1): Sequential(
          (0): Bottleneck(
            (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn1): FrozenBatchNorm2d(64, eps=0.0)
            (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (bn2): FrozenBatchNorm2d(64, eps=0.0)
            (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
            (bn3): FrozenBatchNorm2d(256, eps=0.0)
            (relu): ReLU(inplace=True)
            (downsample): Sequential(
              (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias

# Evaluate baseline performance

In [20]:
model.label_dict = {'cow': 0}  # Assign a unique integer ID to the 'cow' label
model.config['gpus'] = '-1'  # Use GPU (set to '0' for the first GPU or '-1' for all GPUs)

# Set the directory to save the results of the pretrained model
baseline_save_dir = os.path.join(notebook_folder, 'baseline_pred_result')
os.makedirs(baseline_save_dir, exist_ok=True)

# Evaluate the pretrained model on the test set (using test_file)
baseline_results = model.evaluate(test_path, os.path.dirname(test_path), iou_threshold=0.4, savedir=baseline_save_dir)

print("Baseline evaluation complete. Results saved to", baseline_save_dir)

Predicting: |                                                                                                 …

/Users/cam/miniconda3/envs/deepforest/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.16 (you have 1.4.14). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Baseline evaluation complete. Results saved to ./notebook/baseline_pred_result


In [21]:
print("Baseline performance")
# Print box recall and precision with clean formatting
print(f"Box Recall: {baseline_results['box_recall']:.4f}")
print(f"Box Precision: {baseline_results['box_precision']:.4f}")

# Compute and print the mean IoU, rounded to 4 decimal places
mean_iou = np.mean(baseline_results['results']['IoU'])
print(f"Mean IoU: {mean_iou:.4f}")

Box Recall: 0.4405
Box Precision: 0.5826
Mean IoU: 0.3135


# Load finetuned model

In [24]:
# Download the finetuned model checkpoint from Hugging Face
ckpt_path = hf_hub_download(
    repo_id="camappel/deepforest-livestock",  # Replace with your Hugging Face repo ID
    filename="finetuned_checkpoint.ckpt"     # The .ckpt file you uploaded
)

# Load the model checkpoint correctly using the class, not an instance
model = main.deepforest.load_from_checkpoint(ckpt_path)

print("Finetuned model loaded successfully!")

finetuned_checkpoint.ckpt:   0%|          | 0.00/257M [00:00<?, ?B/s]

Reading config file: /Users/cam/miniconda3/envs/deepforest/lib/python3.10/site-packages/deepforest/data/deepforest_config.yml


GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs


Finetuned model loaded successfully!


# Evaluate finetuned performance

In [25]:
model.label_dict = {'cow': 0}  # Assign a unique integer ID to the 'cow' label
model.config['gpus'] = '-1'  # Use GPU (set to '0' for the first GPU or '-1' for all GPUs)


# Set the directory to save the results of the pretrained model
finetuned_save_dir = os.path.join(notebook_folder, 'finetuned_pred_result')
os.makedirs(finetuned_save_dir, exist_ok=True)

# Evaluate the pretrained model on the test set (using test_file)
finetuned_results = model.evaluate(test_path, os.path.dirname(test_path), iou_threshold=0.4, savedir=finetuned_save_dir)

print("Finetuned evaluation complete. Results saved to", finetuned_save_dir)

Predicting: |                                                                                                 …

/Users/cam/miniconda3/envs/deepforest/lib/python3.10/site-packages/albumentations/__init__.py:13: UserWarning: A new version of Albumentations is available: 1.4.16 (you have 1.4.14). Upgrade using: pip install -U albumentations. To disable automatic update checks, set the environment variable NO_ALBUMENTATIONS_UPDATE to 1.
  check_for_updates()


Finetuned evaluation complete. Results saved to ./notebook/finetuned_pred_result


In [27]:
print("Finetuned performance")
# Print box recall and precision with clean formatting
print(f"Box Recall: {finetuned_results['box_recall']:.4f}")
print(f"Box Precision: {finetuned_results['box_precision']:.4f}")

# Compute and print the mean IoU, rounded to 4 decimal places
mean_iou = np.mean(finetuned_results['results']['IoU'])
print(f"Mean IoU: {mean_iou:.4f}")

Finetuned performance
Box Recall: 0.9535
Box Precision: 0.8587
Mean IoU: 0.6571


# Notes

In [ ]:
# List to hold the paths of all downloaded files
downloaded_files = []

# Iterate over all files in the registry dictionary to download them
for zip_file in data.registry.keys():
    file_path = data.fetch(zip_file)
    downloaded_files.append(file_path)
    print(f"Downloaded {zip_file} to {file_path}")

In [ ]:
downloaded_files = []
for zip_file in data.registry.keys():
    file_path = os.path.join(data.abspath, zip_file)  # Get the absolute path for each downloaded file
    downloaded_files.append(file_path)

print(notebook_folder)
print(downloaded_files)

In [ ]:
os.listdir('../../../../../../../OneDrive')

In [ ]:
os.makedirs('../../../../../../../Documents/notebook_combined')

# Combine all parts into a single archive using py7zr
combined_file_path = os.path.join('../../../../../../../Documents/notebook_combined', "Annotated_cows_GenTORE.7z")

with open(combined_file_path, 'wb') as combined_file:
    for part in sorted(downloaded_files):
        with open(part, 'rb') as part_file:
            combined_file.write(part_file.read())

print(combined_file_path)

In [ ]:
# Extract the combined 7z archive
with py7zr.SevenZipFile(combined_file_path, mode='r') as archive:
    archive.extractall(path='../../../../../../../Documents/notebook_combined')

print(f"All files have been extracted to ./notebook_combined'.")

Download and combine without saving

In [ ]:
# Define the notebook folder path
notebook_folder = './notebook'

# Create the notebook folder if it does not exist
if not os.path.exists(notebook_folder):
    os.makedirs(notebook_folder)

# Create a Pooch object for handling data
data = pooch.create(
    base_url="doi:10.7910/DVN/N7GJYU", 
    path=pooch.os_cache("myproject")
)

# Load the registry from the DOI
data.load_registry_from_doi()

# List to hold the paths of all downloaded files
downloaded_files = []

# First loop to download all files
for zip_file in data.registry.keys():
    file_path = data.fetch(zip_file)  # Fetch and download each file
    downloaded_files.append(file_path)  # Collect all downloaded files
    print(f"Downloaded {zip_file} to {file_path}")

print("All parts have been downloaded successfully.")

# Direct extraction from the first part using py7zr
# Ensure all parts (.001, .002, etc.) are present in the same directory
with py7zr.SevenZipFile(downloaded_files[0], mode='r') as archive:
    archive.extractall(path=notebook_folder)

print(f"All files have been extracted to '{notebook_folder}'.")

## Fetch RGB images from Zenodo

Fetch sample images from the publicly accessible [NEON](https://zenodo.org/records/3459803) training set to evaluate DeepForest's detection and classification performance on different types of landscapes

### Small distinct tree-crowns

Find hash code

In [ ]:
fname = "2018_SJER_3_258000_4106000_image.tif"

pooch.retrieve(
    url=f"doi:10.5281/zenodo.3459803/{fname}",
    known_hash="md5:d70ecbee40abe043946e8e492c514a63",
    path=notebook_folder,
    fname=fname
)

In [ ]:
# set catalogue location
catalog_file = os.path.join(notebook_folder, 'catalog.yaml')

with open(catalog_file, 'w') as f:
    f.write(f'''
sources:
  NEONTREE_rgb:
    driver: xarray_image
    description: 'NeonTreeEvaluation RGB images (collection)'
    args:
      urlpath: "{{{{ CATALOG_DIR }}}}/{fname}"
      ''')

Load an intake catalog for the downloaded data.

In [ ]:
cat_tc = intake.open_catalog(catalog_file)

Use intake to load the sample image through dask

In [ ]:
tc_rgb = cat_tc["NEONTREE_rgb"].to_dask()
tc_rgb

Load and prepare labels

In [ ]:
# functions to load xml and extract bounding boxes

# function to create ordered dictionary of .xml annotation files
def loadxml(imagename):
    imagename = imagename.replace('.tif','')
    fullurl = "https://raw.githubusercontent.com/weecology/NeonTreeEvaluation/master/annotations/" + imagename + ".xml"
    file = urllib.request.urlopen(fullurl)
    data = file.read()
    file.close()
    data = xmltodict.parse(data)
    return data

# function to extract bounding boxes
def extractbb(i):
    bb = [f['bndbox'] for f in xml['annotation']['object']]
    return bb

In [ ]:
xml = loadxml(fname)
bball = extractbb(xml)

Visualise image and labels

In [ ]:
# function to plot images
def cv2_imshow(a, **kwargs):
    a = a.clip(0, 255).astype('uint8')
    # cv2 stores colors as BGR; convert to RGB
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)

    return plt.imshow(a, **kwargs)

In [ ]:
image = tc_rgb

# plot predicted bbox
image2 = image.values.copy()
target_bbox = bball
print(type(target_bbox))
print(target_bbox[0:2])

In [ ]:
for row in target_bbox:
    cv2.rectangle(image2, (int(row["xmin"]), int(row["ymin"])), (int(row["xmax"]), int(row["ymax"])), (0,255,255), thickness=2, lineType=cv2.LINE_AA)

plot_reference = plt.figure(figsize=(15,15))
cv2_imshow(np.flip(image2,2))
plt.title('Reference labels',fontsize='xx-large')
plt.show()

### Large dense canopies

In [ ]:
fname = "2018_NIWO_2_450000_4426000_image_crop.tif"

pooch.retrieve(
    url=f"doi:10.5281/zenodo.3459803/{fname}",
    known_hash="md5:c8f700eca920c6f0b93d16e6e26cc5a7",
    path=notebook_folder,
    fname=fname
)

In [ ]:
# set catalogue location
catalog_file = os.path.join(notebook_folder, 'catalog.yaml')

with open(catalog_file, 'w') as f:
    f.write(f'''
sources:
  NEONTREE_rgb:
    driver: xarray_image
    description: 'NeonTreeEvaluation RGB images (collection)'
    args:
      urlpath: "{{{{ CATALOG_DIR }}}}/{fname}"
      ''')

Load an intake catalog for the downloaded data.

In [ ]:
cat_tc = intake.open_catalog(catalog_file)

Use intake to load the sample image through dask

In [ ]:
tc_rgb = cat_tc["NEONTREE_rgb"].to_dask()
tc_rgb

Load and prepare labels

In [ ]:
xml = loadxml(fname)
bball = extractbb(xml)

Visualise image and labels

In [ ]:
# function to plot images
def cv2_imshow(a, **kwargs):
    a = a.clip(0, 255).astype('uint8')
    # cv2 stores colors as BGR; convert to RGB
    if a.ndim == 3:
        if a.shape[2] == 4:
            a = cv2.cvtColor(a, cv2.COLOR_BGRA2RGBA)
        else:
            a = cv2.cvtColor(a, cv2.COLOR_BGR2RGB)

    return plt.imshow(a, **kwargs)

In [ ]:
image = tc_rgb

# plot predicted bbox
image2 = image.values.copy()
target_bbox = bball
print(type(target_bbox))
print(target_bbox[0:2])

In [ ]:
for row in target_bbox:
    cv2.rectangle(image2, (int(row["xmin"]), int(row["ymin"])), (int(row["xmax"]), int(row["ymax"])), (0,255,255), thickness=2, lineType=cv2.LINE_AA)

plot_reference = plt.figure(figsize=(15,15))
cv2_imshow(np.flip(image2,2))
plt.title('Reference labels',fontsize='xx-large')
plt.show()

## Load DeepForest pretrained model

Now we're going to load and use a pretrained model from the deepforest package.

In [ ]:
from deepforest import main

# load deep forest model
model = main.deepforest()
model.use_release()
model.current_device = torch.device("cpu")

In [ ]:
pred_boxes = model.predict_image(image=image.values)
print(pred_boxes.head(5))

In [ ]:
image3 = image.values.copy() 

for index, row in pred_boxes.iterrows():
    cv2.rectangle(image3, (int(row["xmin"]), int(row["ymin"])), (int(row["xmax"]), int(row["ymax"])), (0,255,255), thickness=2, lineType=cv2.LINE_AA)

plot_fullimage = plt.figure(figsize=(15,15))
cv2_imshow(np.flip(image3,2))
plt.title('Full-image predictions',fontsize='xx-large')
plt.show()

## Comparison full image prediction and reference labels
Let's compare the labels and predictions over the tested image.

In [ ]:
plot_referandfullimage = plt.figure(figsize=(15,15))
ax1 = plt.subplot(1, 2, 1), cv2_imshow(np.flip(image2,2))
ax1[0].set_title('Reference labels',fontsize='xx-large')
ax2 = plt.subplot(1, 2, 2), cv2_imshow(np.flip(image3,2))
ax2[0].set_title('Full-image predictions', fontsize='xx-large')
plt.show() # To show figure

## Tile-based prediction
To optimise the predictions, the DeepForest can be run tile-wise.

The following cells show how to define the optimal window i.e. tile size.

In [ ]:
from deepforest import preprocess

#Create windows of 400px
windows = preprocess.compute_windows(image.values, patch_size=400,patch_overlap=0)
print(f'We have {len(windows)} windows in the image')

In [ ]:
#Loop through a few sample windows, crop and predict
plot_tilewindows, axes, = plt.subplots(nrows=2,ncols=2, figsize=(15,15))
axes = axes.flatten()
for index2 in range(4):
    crop = image.values[windows[index2].indices()]
    #predict in bgr channel order, color predictions in red.
    boxes = model.predict_image(image=np.flip(crop[...,::-1],2), return_plot = True)

    #but plot in rgb channel order
    axes[index2].imshow(boxes[...,::-1])
    axes[index2].set_title(f'Prediction in Window {index2 + 1} out of {len(windows)}', fontsize='xx-large')

Once a suitable tile size is defined, we can run in a batch using the predict_tile function:

In [ ]:
tile = model.predict_tile(image=image.values,return_plot=False,patch_overlap=0,iou_threshold=0.05,patch_size=400)

# plot predicted bbox
image_tile = image.values.copy()

for index, row in tile.iterrows():
    cv2.rectangle(image_tile, (int(row["xmin"]), int(row["ymin"])), (int(row["xmax"]), int(row["ymax"])), (0, 255, 255), thickness=2, lineType=cv2.LINE_AA)

plot_tilewise = plt.figure(figsize=(15,15))
ax1 = plt.subplot(1, 2, 1), cv2_imshow(np.flip(image2,2))
ax1[0].set_title('Reference labels',fontsize='xx-large')
ax2 = plt.subplot(1, 2, 2), cv2_imshow(np.flip(image_tile,2))
ax2[0].set_title('Tile-wise predictions', fontsize='xx-large')
plt.show() # To show figure

# Interactive plots

The plot below summarises above static plots by interactively comparing bounding boxes and scores of full-image and tile-wise predictions. To zoom-in the reference NEON RGB image with its original resolution change rasterize=True to rasterize=False.

In [ ]:
## function to convert bbox in dictionary to geopandas
def bbox_to_geopandas(bbox_df):
    geometry = [box(x1, y1, x2, y2) for x1,y1,x2,y2 in zip(bbox_df.xmin, bbox_df.ymin, bbox_df.xmax, bbox_df.ymax)]
    poly_geo = GeoDataFrame(bbox_df, geometry=geometry)
    return poly_geo

## prepare reference and prediction bbox
### convert data types for reference bbox dictionary
reference = pd.DataFrame.from_dict(target_bbox)
reference[['xmin', 'ymin', 'xmax', 'ymax']] = reference[['xmin', 'ymin', 'xmax', 'ymax']].astype(int)

poly_reference = bbox_to_geopandas(reference)
poly_prediction_image = bbox_to_geopandas(pred_boxes)
poly_prediction_tile = bbox_to_geopandas(tile)

## settings for hvplot objects
settings_vector = dict(fill_color=None, width=400, height=400, clim=(0,1), fontsize={'title': '110%'})
settings_image = dict(x='x', y='y', data_aspect=1, xaxis=False, yaxis=None)

## create hvplot objects
plot_RGB = tc_rgb.hvplot.rgb(**settings_image, bands='channel', hover=False, rasterize=True)
plot_vector_reference = poly_reference.hvplot(hover_cols=False, legend=False).opts(title='Reference labels', alpha=1, **settings_vector)
plot_vector_image = poly_prediction_image.hvplot(hover_cols=['score'], legend=False).opts(title='Full-image predictions', alpha=0.5, **settings_vector)
plot_vector_tile = poly_prediction_tile.hvplot(hover_cols=['score'], legend=False).opts(title='Tile-wise predictions', alpha=0.5, **settings_vector)

plot_comparison = pn.Row(pn.Column(plot_RGB * plot_vector_reference, 
                         plot_RGB * plot_vector_image),
                         pn.Column(pn.Spacer(background='white', width=400, height=400),  
                         plot_RGB * plot_vector_tile), scroll=True)

plot_comparison.embed()

In [ ]:
from deepforest.model import CropModel

model = CropModel()

## Summary
*Provide 3-5 bullet points summarising the main aspects of the dataset and tools covered in the notebook.*

* Sentence 1 e.g. `tool-name` to perform...
* Sentence 2 e.g. `tool-name` to perform...

## Additional information
**Dataset**: Type here details of dataset(s) version.

**Codebase**: Type here details of codebase version (only for notebooks categorised under modelling/preprocesing/post-processing themes).

**License**: The code in this notebook is licensed under the MIT License. The Environmental Data Science book is licensed under the Creative Commons by Attribution 4.0 license. See further details [here](https://github.com/alan-turing-institute/environmental-ds-book/blob/master/LICENSE.md).

**Contact**: If you have any suggestion or report an issue with this notebook, feel free to [create an issue](https://github.com/alan-turing-institute/environmental-ds-book/issues/new/choose) or send a direct message to [environmental.ds.book@gmail.com](mailto:environmental.ds.book@gmail.com).

In [ ]:
from datetime import date
print(f'Last tested: {date.today()}')